# O2 ROP Machine Learning Models

## Introduction

This notebook focuses on developing machine learning models for O2 ROP data. The following steps are covered:

1. Data import and processing
2. Feature selection
3. Model development and validation

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble
from sklearn.metrics import roc_auc_score, make_scorer, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from scipy.stats import ttest_ind


# Disable chained assignments
pd.options.mode.chained_assignment = None 

# Suppress FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Scoring metrics
ROC_scorer = make_scorer(roc_auc_score)
F1_scorer = make_scorer(f1_score)
Precision_scorer = make_scorer(precision_score)
Recall_scorer = make_scorer(recall_score)

## Data Import and Processing

In [ ]:
# Input features with 4 variables
X = ohsu_baby_df[['GA', 'Spo2_Hyper_Mean', 'fio2_cov_Mean', 'fio2_Mean']].to_numpy()
# Input with gestational age alone
X_GA = ohsu_baby_df[['GA']].to_numpy()
# Output feature
y = ohsu_baby_df["binary_ROP"].to_numpy()

# Normalize the features
scaler = StandardScaler()

## Feature Selection

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

X = ohsu_baby[['BW', 'GA', 'Gender', 'Spo2_Mean', 'Spo2_cov_Mean', 'Spo2_Hyper_Mean', 'fio2_cov_Mean', 'fio2_Mean',  'cumDaily_fio2']]
y = ohsu_baby["binary_ROP"].to_numpy()

svm = SVC(gamma='scale', class_weight='balanced', kernel ='linear', probability=True, random_state=42)
rf = RandomForestClassifier(n_estimators = 100, max_depth = 4, min_samples_split=5, class_weight='balanced_subsample',
                           min_samples_leaf=3, max_features='sqrt', random_state=42)

# The "accuracy" scoring shows the proportion of correct classifications

min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(
    estimator=rf,
    step=1,
    cv=StratifiedKFold(5),
    scoring="roc_auc",
    min_features_to_select=1,
)
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(
    range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
    rfecv.grid_scores_,
)
plt.show()

TF_list = rfecv.support_
feature_list =  rfecv.feature_names_in_

select_features = [x for x, y in zip(feature_list, TF_list) if y == bool('True')]
len(select_features)

print(select_features)

## Model Development and Validation

* Fine tuning with grid search...

### Data Splitting for Cross-Validation

In [ ]:
# Lets split the data into 5 folds.  
# We will use this 'kf'(KFold splitting stratergy) object as input to cross_val_score() method
kf =KFold(n_splits=5, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

### Random Forest Model

In [ ]:
#Random forest
score = cross_val_score(RandomForestClassifier(n_estimators = 100, max_depth = 4, min_samples_split=5, class_weight='balanced_subsample',
                           min_samples_leaf=3, max_features='sqrt', random_state=42) , X, y, cv=kf, scoring="roc_auc")
print(f'Scores for each fold: {score}')
print(f'Scores for average score: {np.mean(score)}')

### Support Vector Machine (SVM) Model

In [ ]:
#SVM
score2 = cross_val_score(SVC(gamma='scale', class_weight='balanced', kernel ='linear', probability=True, random_state=42) , X, y, cv=kf, scoring="roc_auc")

print(f'Scores for each fold: {score2}')
print(f'Scores for average score: {np.mean(score2)}')


## SVM Model Comparison with Monte Carlo + 5-Fold Cross-Validation

* This technique is often used to assess the variability and robustness of a model's performance

In [ ]:
# Input features with 4 variables
X = ohsu_baby_df[['GA', 'Spo2_Hyper_Mean', 'fio2_cov_Mean', 'fio2_Mean']].to_numpy()
# Input with gestational age alone
X_GA = ohsu_baby_df[['GA']].to_numpy()
# Output feature
y = ohsu_baby_df["binary_ROP"].to_numpy()

# Initialize lists to collect metrics
svm_AUCs_X1 = []
svm_AUPRCs_X1 = []
svm_AUCs_X2 = []
svm_AUPRCs_X2 = []

# Pre-defined sequence of random seeds for reproducibility
random_seeds = np.random.RandomState(42).randint(0, 10000, size=100)

# Perform 100 repeats
for i in range(100):
    random_state = random_seeds[i]
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    svm_model = SVC(gamma='scale', class_weight='balanced', kernel ='linear', probability=True, random_state=42)
    
    # Metrics for X features
    svm_AUC_X1 = cross_val_score(svm_model, X, y, cv=cv, scoring='roc_auc').mean()
    svm_AUPRC_X1 = cross_val_score(svm_model, X, y, cv=cv, scoring='average_precision').mean()
    
    # Metrics for X_GA features
    svm_AUC_X2 = cross_val_score(svm_model, X_GA, y, cv=cv, scoring='roc_auc').mean()
    svm_AUPRC_X2 = cross_val_score(svm_model, X_GA, y, cv=cv, scoring='average_precision').mean()

    # Append to lists
    svm_AUCs_X1.append(svm_AUC_X1)
    svm_AUPRCs_X1.append(svm_AUPRC_X1)
    svm_AUCs_X2.append(svm_AUC_X2)
    svm_AUPRCs_X2.append(svm_AUPRC_X2)

# Calculate averages
avg_svm_AUC_X1 = np.mean(svm_AUCs_X1)
avg_svm_AUPRC_X1 = np.mean(svm_AUPRCs_X1)
avg_svm_AUC_X2 = np.mean(svm_AUCs_X2)
avg_svm_AUPRC_X2 = np.mean(svm_AUPRCs_X2)

# Perform t-tests
t_stat_AUC, p_val_AUC = ttest_ind(svm_AUCs_X1, svm_AUCs_X2)
t_stat_AUPRC, p_val_AUPRC = ttest_ind(svm_AUPRCs_X1, svm_AUPRCs_X2)

print(f"Average AUC for X1: {avg_svm_AUC_X1}, Average AUC for X2: {avg_svm_AUC_X2}")
print(f"Average AUPRC for X1: {avg_svm_AUPRC_X1}, Average AUPRC for X2: {avg_svm_AUPRC_X2}")
print(f"T-test p-value for AUC: {p_val_AUC}")
print(f"T-test p-value for AUPRC: {p_val_AUPRC}")

## Visualizing 5-Fold Cross-Validation Results

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold


# Input features with 4 variables
X = ohsu_baby_df[['GA', 'Spo2_Hyper_Mean', 'fio2_cov_Mean', 'fio2_Mean']].to_numpy()
# Input with gestational age alone
X_GA = ohsu_baby_df[['GA']].to_numpy()
# Output feature
y = ohsu_baby_df["binary_ROP"].to_numpy()

rf_model = RandomForestClassifier(n_estimators = 100, max_depth = 4, min_samples_split=3, class_weight='balanced',
                           min_samples_leaf=5, max_features='sqrt', random_state=42)
svm_model = SVC(gamma='scale', class_weight='balanced', kernel ='linear', probability=True, random_state=42)


# Define the models
models = [
    ('Random Forest', rf_model),
    ('SVM', svm_model)
]

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(8, 6))

# Perform 5-fold cross-validation
cv = StratifiedKFold(n_splits=5)
for name, model in models:
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    for train, test in cv.split(X, y):
        model.fit(X.iloc[train], y.iloc[train])
        probas_ = model.predict_proba(X.iloc[test])[:, 1]
        fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_)
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

    # Plot the mean ROC curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, label=f'{name} (AUC = {mean_auc:.4f} +/- {std_auc:.4f})')

# Plot the random line
ax.plot([0, 1], [0, 1], linestyle='--', color='black')

# Set the labels and title
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver Operating Characteristic')

# Add the legend
ax.legend(loc='lower right')
plt.show()
